In [2]:
import pandas as pd
import numpy as np
import time
import re

from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm
import requests

# path를 자신의 경로에 맞게 지정!
path = "/Users/Goo/대구_빅데이터/"

### Search_content 항목
0. 여행(소비,여행 | 소비,숙소 | 소비,펜션 | 소비,호텔 | 소비,모텔 | 소비,항공 | 소비,택시 | 소비,버스 | 소비,기차 | 소비,교통 | 소비,렌트카)
1. 취미(소비,레져 | 소비,공연 | 소비,골프 | 소비,악기 | 소비,스포츠 | 소비,영화 | 소비,티켓팅)
2. IT_전자_자동차(소비,스마트폰 | 소비,아이폰 | 소비,갤럭시 | 소비,가전제품 | 소비,국산차 | 소비,외제차)
3. 생활(소비,할인 | 소비,마트 | 소비,편의점 | 소비,주유 | 소비,생활용품 | 소비,생필품 | 소비,가구 | 소비,주방용품 )
4. 패션_뷰티(소비,의류 | 소비,패션 | 소비,뷰티 | 소비,패션잡화 | 소비,화장품 | 소비,미용 | 소비,백화점 )
5. 교육(소비,교육 | 소비,문제집 | 소비,학원 | 소비,도서 | 소비,교재 | 소비,입시 | 소비,수능 | 소비,내신 | 소비,학종)
6. 의료(소비,병원 | 소비,약국 | 소비,건강식품 | 소비,보험 | 소비,한약 | 소비,진료)
7. 외식(소비,외식 | 소비,간편식 | 소비,한식 | 소비,중식 | 소비,양식 | 소비,일식 | 소비,주점 | 소비,제과점)

연도 기준 : 18년 6월부터 23년 5월까지의 뉴스 데이터  
봄 : 3-5월, 여름 : 6-8월, 가을 9-11월, 겨울 12-2월  
-> 페이지 수로 뉴스를 가져왔을때 업종별, 계절별로 크롤링 해올 수 있는 뉴스의 개수가 모두 달라서 최신년도부터 계절별로 300개씩 크롤링해와 업종별 1200개의 소비자 뉴스를 통해 계절성을 반영할 수 있도록 학습데이터를 구축하였습니다. ex) 특정 계절의 23년도 뉴스를 모두 크롤링 해왔는데 300개가 되지 않았으면 22년도 해당 계절의 뉴스를 이어서 크롤링 해옴.  

URL Format : https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_content}&start={page}&pd=3&ds={start_day}&de={end_day}  
-조건 : start_day, end_day = "yyyy.mm.dd"형식, page는 1, 11, 21형식으로 페이지마다 10씩 늘어남

Features : 신문사, 링크, 제목, 원문, label

In [2]:
# 필요한 키워드 입력
search_content = input("검색할 키워드를 입력해주세요. ex) 축구 | 농구 | 야구('|' 사이 공백 필수) : ")
max_news = int(input("\n몇 개의 뉴스를 크롤링할지 입력해주세요. ex) 1000(숫자만입력): "))
sector = input("\n업종명을 입력해주세요. ex) 여행, 취미, IT_전자_자동차, 생활, ...: ")
label = int(input("\nLabel을 입력해주세요. ex) 0, 1, 2, 3, ...: "))

검색할 키워드를 입력해주세요. ex) 축구 | 농구 | 야구('|' 사이 공백 필수) : 소비,병원 | 소비,약국 | 소비,건강식품 | 소비,보험 | 소비,한약 | 소비,진료

몇 개의 뉴스를 크롤링할지 입력해주세요. ex) 1000(숫자만입력): 300

업종명을 입력해주세요. ex) 여행, 취미, IT_전자_자동차, 생활, ...: 의료

Label을 입력해주세요. ex) 0, 1, 2, 3, ...: 6


In [3]:
# 4계절 크롤링할 기간 설정
spring_startday_list = ["2023.03.01", "2022.03.01", "2021.03.01", "2020.03.01", "2019.03.01"]
spring_endday_list = ["2023.05.31", "2022.05.31", "2021.05.31", "2020.05.31", "2019.05.31"]

summer_startday_list = ["2022.06.01", "2021.06.01", "2020.06.01", "2019.06.01", "2018.06.01"]
summer_endday_list = ["2022.08.31", "2021.08.31", "2020.08.31", "2019.08.31", "2018.08.31"]

fall_startday_list = ["2022.09.01", "2021.09.01", "2020.09.01", "2019.09.01", "2018.09.01"]
fall_endday_list = ["2022.11.30", "2021.11.30", "2020.11.30", "2019.11.30", "2018.11.30"]

winter_startday_list = ["2022.12.01", "2021.12.01", "2020.12.01","2019.12.01", "2018.12.01"]
winter_endday_list = ["2023.02.28", "2022.02.28", "2021.02.28", "2020.02.29", "2019.02.28"]

In [4]:
print("검색 항목 : ", search_content)
print("계절별 최대 뉴스 개수 : ", max_news)
print("업종명 : ", sector)
print("Label : ", label)

검색 항목 :  소비,병원 | 소비,약국 | 소비,건강식품 | 소비,보험 | 소비,한약 | 소비,진료
계절별 최대 뉴스 개수 :  300
업종명 :  의료
Label :  6


In [5]:
# URL 계절별 crawling 함수
# -> Redirect 되지않는 네이버뉴스 max_news개가 추출될때까지 크롤링을 계속하는 함수

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

def url_crawling(search_content, start_season, end_season, max_news):
    
    # 집합 형태로 저장해 중복 url 제거
    url_set = set()
    for start_day, end_day in zip(start_season, end_season):    
        for page in tqdm(range(1, 10000, 10)):
            response = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_content}&start={page}&pd=3&ds={start_day}&de={end_day}", headers=headers)
            # page를 넘기다 page가 없으면 종료
            # 200은 HTTP 상태코드중 하나로 OK의 의미를 가짐. 요청이 성공적으로 처리되었음을 나타냄. 200이 아니라는것은 페이지가 없어 페이지를 넘길 수 없다는 의미
            if response.status_code != 200:
                print(f"페이지 {page//10}가 없습니다. Exiting.")
                break
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            ul = soup.select_one("div.group_news > ul.list_news")

            if ul is None:
                break
            li_list = ul.find_all('li')
            for li in li_list:
                a_list = li.select('div.news_area > div.news_info > div.info_group > a.info')
                for a_tag in a_list:
                    href = a_tag.get('href')
                    # href 속성값이 "n.news.naver.com"(네이버 뉴스)을 포함하는지 확인한다.
                    if "n.news.naver.com" in href:
                        try:
                            # request.head()로 추출한 url이 rediret되는지 확인한다. redirect 되지않은 url만 저장한다.
                            response = requests.head(href, allow_redirects=True)
                            if response.status_code == 200:
                                url_set.add(href)
                                # 계절별 300개의 기사가 모두 크롤링 되었으면 크롤링 종료
                                if len(url_set) >= max_news:
                                    return list(url_set)
                        except Exception as e:
                            print(f"An error occurred: {e}")
            time.sleep(1)

    return list(url_set)

In [6]:
url_spring = url_crawling(search_content, spring_startday_list, spring_endday_list, max_news)
len(url_spring)

 15%|██████                                  | 153/1000 [04:38<25:39,  1.82s/it]


300

In [7]:
url_summer = url_crawling(search_content, summer_startday_list, summer_endday_list, max_news)
len(url_summer)

 17%|██████▊                                 | 169/1000 [05:14<25:46,  1.86s/it]


300

In [8]:
url_fall = url_crawling(search_content, fall_startday_list, fall_endday_list, max_news)
len(url_fall)

 16%|██████▍                                 | 160/1000 [05:06<26:48,  1.92s/it]


300

In [9]:
url_winter = url_crawling(search_content, winter_startday_list, winter_endday_list, max_news)
len(url_winter)

 16%|██████▍                                 | 162/1000 [05:03<26:09,  1.87s/it]


300

In [16]:
news_url = url_spring + url_summer + url_fall + url_winter

In [17]:
# 신문사, 제목, 본문 추출
news_company = []
news_title = []
news_content = []

for url in tqdm(news_url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    company = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_top > a > img[alt]")
    news_company.append(company['alt'] if company else 'None')
    title = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    news_title.append(title.text if title else 'None')
    content = soup.select_one("article#dic_area")
    news_content.append(content.text if content else 'None')

100%|███████████████████████████████████████| 1200/1200 [02:44<00:00,  7.32it/s]


In [18]:
# 데이터프레임 생성
columns = ["company", "url", "title", "content", "label"]

data = {
    "company": news_company,
    "url": news_url,
    "title": news_title,
    "content": news_content,
    "label" : [label] * len(news_company)
}

learn_news = pd.DataFrame(data, columns=columns)
learn_news.head()

,company,url,title,content,label
0,조세일보,https://n.news.naver.com/mnews/article/123/000...,"동아제약, 챔프시럽 '빨간색'만 회수…""책임 다할 것""","\n- 챔프 이부펜 시럽 등 문제없어…소비자 환불 등 개시- ""챔프는 빨간색(아세트...",6
1,이데일리,https://n.news.naver.com/mnews/article/018/000...,'손해사정사 선임권' 알고 계셨나요?…보험사 안내문 뜯어보니,"\n'보험사 셀프 논란 방지' 손해사정사 선임권, 법제화 3년""달라진게 없다"" 국내...",6
2,한국경제,https://n.news.naver.com/mnews/article/015/000...,시늉만 내는 '비대면 초진',"\n복지부, 6월 1일부터 시범사업 시행소아과, 야간·휴일 상담만 가능…처방은 못 ...",6
3,데일리안,https://n.news.naver.com/mnews/article/119/000...,[데일리안 주간 포토] 누리호 3차 발사 성공! 대한민국 우주개발 시대 열렸다!,"\n\n\n\n\nⓒ데일리안[데일리안 = 홍금표기자, 박항구 기자] \n\n\n\n...",6
4,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,"""사무장 병원 감시 강화된다""…건보공단이 보건‧의료생협 실질 감독",\n\n\n\n\n자료사진.연합뉴스 [파이낸셜뉴스] 보건·의료생협에 대한 국민건강...,6


In [19]:
len(learn_news)

1200

In [22]:
# 내용 기준 중복 뉴스 제거
learn_news = learn_news.drop_duplicates(subset=['content'], keep='first')

In [23]:
len(learn_news)

1199

re.sub() : 정규식을 사용하여 문자열 내에서 특정 패턴을 찾아 다른 문자열로 대체  

re.sub(pattern, repl, string)      
-pattern: 찾고자 하는 정규식 패턴을 지정합니다.  
-repl: 대체할 문자열을 지정합니다.  
-string: 대상 문자열을 지정합니다.  

In [24]:
# Escape문자 제거
learn_news['content'] = learn_news['content'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [25]:
learn_news.head()

,company,url,title,content,label
0,조세일보,https://n.news.naver.com/mnews/article/123/000...,"동아제약, 챔프시럽 '빨간색'만 회수…""책임 다할 것""","- 챔프 이부펜 시럽 등 문제없어…소비자 환불 등 개시- ""챔프는 빨간색(아세트아...",6
1,이데일리,https://n.news.naver.com/mnews/article/018/000...,'손해사정사 선임권' 알고 계셨나요?…보험사 안내문 뜯어보니,"'보험사 셀프 논란 방지' 손해사정사 선임권, 법제화 3년""달라진게 없다"" 국내 ...",6
2,한국경제,https://n.news.naver.com/mnews/article/015/000...,시늉만 내는 '비대면 초진',"복지부, 6월 1일부터 시범사업 시행소아과, 야간·휴일 상담만 가능…처방은 못 받...",6
3,데일리안,https://n.news.naver.com/mnews/article/119/000...,[데일리안 주간 포토] 누리호 3차 발사 성공! 대한민국 우주개발 시대 열렸다!,"ⓒ데일리안[데일리안 = 홍금표기자, 박항구 기자] ⓒ데일리안 박항구 기자박민식 국...",6
4,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,"""사무장 병원 감시 강화된다""…건보공단이 보건‧의료생협 실질 감독",자료사진.연합뉴스 [파이낸셜뉴스] 보건·의료생협에 대한 국민건강보험공단의 관리 범...,6


In [26]:
learn_news.to_csv(path+sector+f"_{label}.csv", index=False)

In [17]:
pd.read_csv(path+"의료_6.csv").head(8)

,company,url,title,content,label
0,조세일보,https://n.news.naver.com/mnews/article/123/000...,"동아제약, 챔프시럽 '빨간색'만 회수…""책임 다할 것""","- 챔프 이부펜 시럽 등 문제없어…소비자 환불 등 개시- ""챔프는 빨간색(아세트아...",6
1,이데일리,https://n.news.naver.com/mnews/article/018/000...,'손해사정사 선임권' 알고 계셨나요?…보험사 안내문 뜯어보니,"'보험사 셀프 논란 방지' 손해사정사 선임권, 법제화 3년""달라진게 없다"" 국내 ...",6
2,한국경제,https://n.news.naver.com/mnews/article/015/000...,시늉만 내는 '비대면 초진',"복지부, 6월 1일부터 시범사업 시행소아과, 야간·휴일 상담만 가능…처방은 못 받...",6
3,데일리안,https://n.news.naver.com/mnews/article/119/000...,[데일리안 주간 포토] 누리호 3차 발사 성공! 대한민국 우주개발 시대 열렸다!,"ⓒ데일리안[데일리안 = 홍금표기자, 박항구 기자] ⓒ데일리안 박항구 기자박민식 국...",6
4,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,"""사무장 병원 감시 강화된다""…건보공단이 보건‧의료생협 실질 감독",자료사진.연합뉴스 [파이낸셜뉴스] 보건·의료생협에 대한 국민건강보험공단의 관리 범...,6
5,스포츠동아,https://n.news.naver.com/mnews/article/382/000...,‘커피와 단백질을 한번에’…K&C 헬스바이오 단백질 커피 ‘프로루틴’ 출시,K&C헬스바이오는 커피를 즐기면서 단백질도 보충할 수 있는 고단백 음료 ‘프로루틴...,6
6,디지털타임스,https://n.news.naver.com/mnews/article/029/000...,"[오늘의DT인] 판매왕·임원까지 거친 `보험고수`… ""모르면 차라리 가입하지 마세요""",보험사는 고객이 1~2년내 해지하면 완납유지보다 더 수익 높아지는 구조오래된 보험...,6
7,매일경제,https://n.news.naver.com/mnews/article/009/000...,"너도나도 뛰어드는 펫 영양제 시장 ""전문성 갖춰야 반려인 마음 잡죠""","'반려동물이 곧 가족' 인식 확산간식 하나도 성분 따져 구입펫 영양제, 차별화 전...",6


In [13]:
pd.set_option("display.max_rows", None)  # 모든 행 표시
pd.set_option("display.max_columns", None)  # 모든 열 표시

pd.set_option("display.max_rows", 10)  # 최대 10개의 행만 표시


In [15]:
import pandas as pd

data = [
    ("여행", "소비,여행 | 소비,숙소 | 소비,펜션 | 소비,호텔 | 소비,모텔 | 소비,항공 | 소비,택시 | 소비,버스 | 소비,기차 | 소비,교통 | 소비,렌트카"),
    ("취미", "소비,레져 | 소비,공연 | 소비,골프 | 소비,악기 | 소비,스포츠 | 소비,영화 | 소비,티켓팅"),
    ("IT_전자_자동차", "소비,스마트폰 | 소비,아이폰 | 소비,갤럭시 | 소비,가전제품 | 소비,국산차 | 소비,외제차"),
    ("생활", "소비,할인 | 소비,마트 | 소비,편의점 | 소비,주유 | 소비,생활용품 | 소비,생필품 | 소비,가구 | 소비,주방용품"),
    ("패션_뷰티", "소비,의류 | 소비,패션 | 소비,뷰티 | 소비,패션잡화 | 소비,화장품 | 소비,미용 | 소비,백화점"),
    ("교육", "소비,교육 | 소비,문제집 | 소비,학원 | 소비,도서 | 소비,교재 | 소비,입시 | 소비,수능 | 소비,내신 | 소비,학종"),
    ("의료", "소비,병원 | 소비,약국 | 소비,건강식품 | 소비,보험 | 소비,한약 | 소비,진료"),
    ("외식", "소비,외식 | 소비,간편식 | 소비,한식 | 소비,중식 | 소비,양식 | 소비,일식 | 소비,주점 | 소비,제과점")
]

# 데이터 가공 및 데이터프레임 생성


df = pd.DataFrame(data, columns=["업종분류", "검색 키워드"])

df

,업종분류,검색 키워드
0,여행,"소비,여행 | 소비,숙소 | 소비,펜션 | 소비,호텔 | 소비,모텔 | 소비,항공 ..."
1,취미,"소비,레져 | 소비,공연 | 소비,골프 | 소비,악기 | 소비,스포츠 | 소비,영화..."
2,IT_전자_자동차,"소비,스마트폰 | 소비,아이폰 | 소비,갤럭시 | 소비,가전제품 | 소비,국산차 |..."
3,생활,"소비,할인 | 소비,마트 | 소비,편의점 | 소비,주유 | 소비,생활용품 | 소비,..."
4,패션_뷰티,"소비,의류 | 소비,패션 | 소비,뷰티 | 소비,패션잡화 | 소비,화장품 | 소비,..."
5,교육,"소비,교육 | 소비,문제집 | 소비,학원 | 소비,도서 | 소비,교재 | 소비,입시..."
6,의료,"소비,병원 | 소비,약국 | 소비,건강식품 | 소비,보험 | 소비,한약 | 소비,진료"
7,외식,"소비,외식 | 소비,간편식 | 소비,한식 | 소비,중식 | 소비,양식 | 소비,일식..."


In [ ]:
da = pd.DataFrame(data={})